In [8]:
import pyodbc 
import geopandas
import pandas
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
import os

Creditals CSV and JSON upload functions

In [ ]:
def create_creditals(credential_name, azure_blob_sercret, cursor):
    try:
        sql_query=f"""REATE DATABASE SCOPED CREDENTIAL {credential_name}
    WITH IDENTITY = 'SHARED ACCESS SIGNATURE',
    SECRET = '{azure_blob_sercret}';"""
        cursor.excute(sql_query)
        cursor.commit()
    except Exception as e:
        print(e)
def create_external_data_source(credential_name ,azure_url, source_name, cursor, source_type='BLOB_STORAGE'):
    try:
        sql_query = f"""CREATE EXTERNAL DATA SOURCE {source_name}
        WITH (
            TYPE = {source_type},
            LOCATION = '{azure_url}',
            CREDENTIAL = {credential_name}
        );"""
        cursor.excute(sql_query)
        cursor.commit()
    except Exception as e:
        print(e)
def create_connection(server, database, username, password, driver='SQL Server'):
    try:
        cnxn = pyodbc.connect('DRIVER={'+driver+'};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
        cursor = cnxn.cursor()
        cursor.execute("SELECT @@version;") 
        cursor.commit
        return cursor
    except Exception as e:
        print(e)
def insert_flatefile_azure(file_name, source_name, table_name, cursor, file_format='CSV'):
    try:
        sql_query = f"""BULK INSERT {table_name}
            FROM '{file_name}'
            WITH (DATA_SOURCE = '{source_name}',
                FORMAT = '{file_format}');"""
        cursor.excute(sql_query)
        cursor.commit()
    except Exception as e:
        print(e)

def create_json_table(table_name, cursor, idx_key=None, idx=name=None, col_store=None):
    sql_query_basic=f"""create table {table_name} (
            _id bigint primary key identity,
            dict nvarchar(max)
    );
    ALTER TABLE {table_name}
    ADD CONSTRAINT [Log record should be formatted as JSON]
                   CHECK (ISJSON(dict)=1);"""
    sql_query_colStore=f"""create sequence {table_name}ID as bigint;
            go
            create table {table_name} (
                _id bigint default(next value for {table_name}ID),
                dict nvarchar(max),

                INDEX cci CLUSTERED COLUMNSTORE
            );"""
    sql_query_idx = f"""create table {table_name} (
    _id bigint primary key identity,
    log nvarchar(max),

    severity AS JSON_VALUE(log, '$.{idx_key}'),
    index ix_severity ({idx_key})
);"""



Azure Helper Functions

In [9]:
def azure_connect():
    try:
        connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
    except Exception as e:
        print(e)
def upload_azure_blob(file_name, container_name, local_path=None):
    upload_file_path = os.path.join(local_path, file_name)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
    with open(upload_file_path, "rb") as data:
        blob_client.upload_blob(data)

SQL Graph

In [ ]:
def create_node_table(node_name, schema):
    sql_query = f"""
    CREATE TABLE {node_name} (
            ID INTEGER PRIMARY KEY, 
            {schema}
    ) AS NODE;
    """

def create_edge_table(edge_name, schema):
    sql_query = f"""
    CREATE TABLE {edge_name} (
        id INTEGER PRIMARY KEY,
        {schema}
    ) AS EDGE;
    """
def insert_edge(filer_value, node_a, node_b, edge_name, node_name, value):
    sql_query = f"""INSERT INTO {edge_name} VALUES ((SELECT $node_id FROM {node_name} WHERE {filer_value} = '{node_a}'),
        (SELECT $node_id FROM {node_name} WHERE {filer_value} = '{node_b}'), {value});"""

GEO data